# Pickle CICIDS 2017

This notebooks intended use is to load the CSV data into a Pandas dataframe, normalize and scale the data, then write the DataFrame into a pickle to save these steps for every ML framework run.  
The output are two pickle files: cic_test_data and cic_test_labels.  
**TODO**: Split these into training and test in a meaningful way! (Most likely by hand?)
These pickles can be restored as dataframes by calling [pandas.read_pickle()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_pickle.html).  
**Hint**: If your are missing the *_clean* CSVs, try running the notebook *Data Sanitazation.ipynb*

In [1]:
import os
import json
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from sklearn import preprocessing
pd.set_option('display.max_columns', None)

Using TensorFlow backend.


## Data Loading and Prep

As there is literally "Inifnity" written in the CSV dataset, we set an additional filter so that these will be replaced by a NaN-representation (that will lateron be set to zero).  
Also, the external_ip field is set to 0.0.0.0 if either NaN or non existent.  
Finally, the dtype for the external_ip column had to be set manually to object as Pandas kept getting confused.

In [2]:
cic_data = pd.DataFrame()

datafile_names_sorted = [
    'Monday-WorkingHours.pcap_ISCX_clean.csv',
    'Tuesday-WorkingHours.pcap_ISCX_clean.csv',
    'Wednesday-WorkingHours.pcap_ISCX_clean.csv',
    'Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX_clean.csv',
    'Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX_clean.csv',
    'Friday-WorkingHours-Morning.pcap_ISCX_clean.csv',
    'Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX_clean.csv',
    'Friday-WorkingHours-Afternoon-DDos.pcap_ISCX_clean.csv'
]

for filename in datafile_names_sorted:
    inputFileName = os.path.join('CICIDS2017', filename)
    print('Appending', inputFileName)
    
    new_flows = pd.read_csv(
        inputFileName,
        na_values="Infinity",
        dtype={'external_ip':'object'},
        parse_dates=['timestamp']
    )
    
    # as this field is not in all flows, double check for it
    if 'external_ip' not in new_flows:
            new_flows['external_ip'] = "0.0.0.0"
    new_flows['external_ip'].fillna("0.0.0.0", inplace=True)
    
    cic_data = cic_data.append(new_flows,ignore_index=True,sort=False)

print('Found these class labels:', str(cic_data.label.unique()))

Appending CICIDS2017\Monday-WorkingHours.pcap_ISCX_clean.csv
Appending CICIDS2017\Tuesday-WorkingHours.pcap_ISCX_clean.csv
Appending CICIDS2017\Wednesday-WorkingHours.pcap_ISCX_clean.csv
Appending CICIDS2017\Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX_clean.csv
Appending CICIDS2017\Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX_clean.csv
Appending CICIDS2017\Friday-WorkingHours-Morning.pcap_ISCX_clean.csv
Appending CICIDS2017\Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX_clean.csv
Appending CICIDS2017\Friday-WorkingHours-Afternoon-DDos.pcap_ISCX_clean.csv
Found these class labels: ['BENIGN' 'FTPPatator' 'SSHPatator' 'DoSSlowloris' 'DoSSlowhttptest'
 'DoSHulk' 'DoSGoldenEye' 'Heartbleed' 'BruteForce' 'XSS' 'SQLInjection'
 'Infiltration' 'Bot' 'PortScan' 'DDoS']


In [3]:
cic_data.fillna(value=0, inplace=True)

In [4]:
print(cic_data.isnull().values.any())

False


In [5]:
cic_data.head()

,flow_id,source_ip,source_port,destination_ip,destination_port,protocol,timestamp,flow_duration,total_fwd_packets,total_backward_packets,total_length_of_fwd_packets,total_length_of_bwd_packets,fwd_packet_length_max,fwd_packet_length_min,fwd_packet_length_mean,fwd_packet_length_std,bwd_packet_length_max,bwd_packet_length_min,bwd_packet_length_mean,bwd_packet_length_std,flow_bytes_per_s,flow_packets_per_s,flow_iat_mean,flow_iat_std,flow_iat_max,flow_iat_min,fwd_iat_total,fwd_iat_mean,fwd_iat_std,fwd_iat_max,fwd_iat_min,bwd_iat_total,bwd_iat_mean,bwd_iat_std,bwd_iat_max,bwd_iat_min,fwd_psh_flags,bwd_psh_flags,fwd_urg_flags,bwd_urg_flags,fwd_header_length,bwd_header_length,fwd_packets_per_s,bwd_packets_per_s,min_packet_length,max_packet_length,packet_length_mean,packet_length_std,packet_length_variance,fin_flag_count,syn_flag_count,rst_flag_count,psh_flag_count,ack_flag_count,urg_flag_count,cwe_flag_count,ece_flag_count,down_per_up_ratio,average_packet_size,avg_fwd_segment_size,avg_bwd_segment_size,fwd_header_length.1,fwd_avg_bytes_per_bulk,fwd_avg_packets_per_bulk,fwd_avg_bulk_rate,bwd_avg_bytes_per_bulk,bwd_avg_packets_per_bulk,bwd_avg_bulk_rate,subflow_fwd_packets,subflow_fwd_bytes,subflow_bwd_packets,subflow_bwd_bytes,init_win_bytes_forward,init_win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,active_mean,active_std,active_max,active_min,idle_mean,idle_std,idle_max,idle_min,label,external_ip
0,192.168.10.5-8.254.250.126-49188-80-6,8.254.250.126,80,192.168.10.5,49188,6,2017-07-03 08:55:58,4,2,0,12.0,0.0,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,3000000.0,5.000000e+05,4.0,0.0,4.0,4.0,4.0,4.0,0.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,40,0,5.000000e+05,0.0,6.0,6.0,6.0,0.0,0.0,0,0,0,0,1,1,0,0,0.0,9.0,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,329,-1,1,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,0.0.0.0
1,192.168.10.5-8.254.250.126-49188-80-6,8.254.250.126,80,192.168.10.5,49188,6,2017-07-03 08:55:58,1,2,0,12.0,0.0,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,12000000.0,2.000000e+06,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,40,0,2.000000e+06,0.0,6.0,6.0,6.0,0.0,0.0,0,0,0,0,1,1,0,0,0.0,9.0,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,329,-1,1,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,0.0.0.0
2,192.168.10.5-8.254.250.126-49188-80-6,8.254.250.126,80,192.168.10.5,49188,6,2017-07-03 08:55:58,1,2,0,12.0,0.0,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,12000000.0,2.000000e+06,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,40,0,2.000000e+06,0.0,6.0,6.0,6.0,0.0,0.0,0,0,0,0,1,1,0,0,0.0,9.0,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,329,-1,1,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,0.0.0.0
3,192.168.10.5-8.254.250.126-49188-80-6,8.254.250.126,80,192.168.10.5,49188,6,2017-07-03 08:55:58,1,2,0,12.0,0.0,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,12000000.0,2.000000e+06,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,40,0,2.000000e+06,0.0,6.0,6.0,6.0,0.0,0.0,0,0,0,0,1,1,0,0,0.0,9.0,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,329,-1,1,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,0.0.0.0
4,192.168.10.14-8.253.185.121-49486-80-6,8.253.185.121,80,192.168.10.14,49486,6,2017-07-03 08:56:22,3,2,0,12.0,0.0,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,4000000.0,6.666667e+05,3.0,0.0,3.0,3.0,3.0,3.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,40,0,6.666667e+05,0.0,6.0,6.0,6.0,0.0,0.0,0,0,0,0,1,1,0,0,0.0,9.0,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,245,-1,1,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,0.0.0.0


## Data Encoding

There's still a problem: How can we encode IP addresses in a way that the neural network can make use of them while preserving the hierarchical information they contain?  
Encoding IPs through One Hot lets comlexity and training times explode, so for now I am splitting each IP into its four octet pairs and interpret them as numbers.  
Maybe there's a better way to represent them (especially because I am only able to encode IPv4 right now).  

**Important**: If this breaks, you forgot to remove the broken external IP in Friday DDoS @ 2017-07-07T15:58:00,26794

In [6]:
# https://stackoverflow.com/questions/14745022/how-to-split-a-column-into-two-columns
# FIXME: Right now, only IPv4 (4 octets)

# Split the String representation of the IP into it's four octects, which are delimited by a dot
cic_data['source_ip_o1'],cic_data['source_ip_o2'],cic_data['source_ip_o3'],cic_data['source_ip_o4'] = cic_data['source_ip'].str.split('.').str
cic_data['destination_ip_o1'],cic_data['destination_ip_o2'],cic_data['destination_ip_o3'],cic_data['destination_ip_o4'] = cic_data['destination_ip'].str.split('.').str
cic_data['external_ip_o1'],cic_data['external_ip_o2'],cic_data['external_ip_o3'],cic_data['external_ip_o4'] = cic_data['external_ip'].str.split('.').str

# After completion, drop the initial columns, as they aren't needed anymore
cic_data.drop(['source_ip'], axis=1, inplace=True)
cic_data.drop(['destination_ip'], axis=1, inplace=True)
cic_data.drop(['external_ip'], axis=1, inplace=True)

In [7]:
# as we're going to normalize the dataset, we should drop the flow_id, as this info cannot be normalized.
cic_data.drop(['flow_id'], axis=1, inplace=True)

# Finally, let's inspect the outcome
cic_data.head()

,source_port,destination_port,protocol,timestamp,flow_duration,total_fwd_packets,total_backward_packets,total_length_of_fwd_packets,total_length_of_bwd_packets,fwd_packet_length_max,fwd_packet_length_min,fwd_packet_length_mean,fwd_packet_length_std,bwd_packet_length_max,bwd_packet_length_min,bwd_packet_length_mean,bwd_packet_length_std,flow_bytes_per_s,flow_packets_per_s,flow_iat_mean,flow_iat_std,flow_iat_max,flow_iat_min,fwd_iat_total,fwd_iat_mean,fwd_iat_std,fwd_iat_max,fwd_iat_min,bwd_iat_total,bwd_iat_mean,bwd_iat_std,bwd_iat_max,bwd_iat_min,fwd_psh_flags,bwd_psh_flags,fwd_urg_flags,bwd_urg_flags,fwd_header_length,bwd_header_length,fwd_packets_per_s,bwd_packets_per_s,min_packet_length,max_packet_length,packet_length_mean,packet_length_std,packet_length_variance,fin_flag_count,syn_flag_count,rst_flag_count,psh_flag_count,ack_flag_count,urg_flag_count,cwe_flag_count,ece_flag_count,down_per_up_ratio,average_packet_size,avg_fwd_segment_size,avg_bwd_segment_size,fwd_header_length.1,fwd_avg_bytes_per_bulk,fwd_avg_packets_per_bulk,fwd_avg_bulk_rate,bwd_avg_bytes_per_bulk,bwd_avg_packets_per_bulk,bwd_avg_bulk_rate,subflow_fwd_packets,subflow_fwd_bytes,subflow_bwd_packets,subflow_bwd_bytes,init_win_bytes_forward,init_win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,active_mean,active_std,active_max,active_min,idle_mean,idle_std,idle_max,idle_min,label,source_ip_o1,source_ip_o2,source_ip_o3,source_ip_o4,destination_ip_o1,destination_ip_o2,destination_ip_o3,destination_ip_o4,external_ip_o1,external_ip_o2,external_ip_o3,external_ip_o4
0,80,49188,6,2017-07-03 08:55:58,4,2,0,12.0,0.0,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,3000000.0,5.000000e+05,4.0,0.0,4.0,4.0,4.0,4.0,0.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,40,0,5.000000e+05,0.0,6.0,6.0,6.0,0.0,0.0,0,0,0,0,1,1,0,0,0.0,9.0,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,329,-1,1,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,8,254,250,126,192,168,10,5,0,0,0,0
1,80,49188,6,2017-07-03 08:55:58,1,2,0,12.0,0.0,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,12000000.0,2.000000e+06,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,40,0,2.000000e+06,0.0,6.0,6.0,6.0,0.0,0.0,0,0,0,0,1,1,0,0,0.0,9.0,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,329,-1,1,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,8,254,250,126,192,168,10,5,0,0,0,0
2,80,49188,6,2017-07-03 08:55:58,1,2,0,12.0,0.0,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,12000000.0,2.000000e+06,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,40,0,2.000000e+06,0.0,6.0,6.0,6.0,0.0,0.0,0,0,0,0,1,1,0,0,0.0,9.0,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,329,-1,1,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,8,254,250,126,192,168,10,5,0,0,0,0
3,80,49188,6,2017-07-03 08:55:58,1,2,0,12.0,0.0,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,12000000.0,2.000000e+06,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,40,0,2.000000e+06,0.0,6.0,6.0,6.0,0.0,0.0,0,0,0,0,1,1,0,0,0.0,9.0,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,329,-1,1,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,8,254,250,126,192,168,10,5,0,0,0,0
4,80,49486,6,2017-07-03 08:56:22,3,2,0,12.0,0.0,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,4000000.0,6.666667e+05,3.0,0.0,3.0,3.0,3.0,3.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,40,0,6.666667e+05,0.0,6.0,6.0,6.0,0.0,0.0,0,0,0,0,1,1,0,0,0.0,9.0,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,245,-1,1,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,8,253,185,121,192,168,10,14,0,0,0,0


Now that this is out of the way, we still need to encode the labels column to numeric values.  
To do this, I'm going to be using the Keras Tokenizer.  
The labels of the dataset (as in: *Benign*, *DDoS*, *Portscan*, etc) are converted into a list of integers and split off of the main DataFrame.  
After this step there is a variable `cic_labels` that holds an integer-encoded list of labels.  
A humble example (not representative):  

|Label         | Value          |
|------------- |---------:|
|Benign      | 0|
|DDoS        | 1|
|Portscan    | 2|  

So if the order of the first three Netflows would be *Benign*, *Benign*, *DDos*,  
the resulting `enc_labels` would look like this: `[1,1,2]`

In [8]:
cic_labels = cic_data.filter(['label'])
cic_data.drop(['label'], axis=1, inplace=True)

In [9]:
number_of_classes = len(cic_labels['label'].unique())

# FIXME: don't fit the tokenizer on the full ds! Split off a test set first!

# tokenize the LABELS
label_tokenizer = Tokenizer(num_words=number_of_classes+1, filters='') # don't filter any of the characters. 1 entry = 1 label 
label_tokenizer.fit_on_texts(cic_labels['label'].unique())

# Run the fitted tokenizer on the label column and save the encoded data as dataframe
enc_labels = label_tokenizer.texts_to_sequences(cic_labels['label'])

# finally, append the encoded labels to the label dataframe
cic_labels = pd.concat([cic_labels, pd.DataFrame(columns=['label_encoded'],dtype=np.int8,data=enc_labels)], axis=1)
cic_labels.head()

,label,label_encoded
0,BENIGN,1
1,BENIGN,1
2,BENIGN,1
3,BENIGN,1
4,BENIGN,1


To be able to translate the encoded labels back, write the Tokenizer wordlist to a file near the CSVs.

In [10]:
filename = os.path.join('CICIDS2017','cic_label_wordindex.json')
print('Writing encoder data to file {}: {}'.format(filename, label_tokenizer.word_index))
with open(filename, 'w') as outfile:
    json.dump(label_tokenizer.word_index, outfile)

Writing encoder data to file CICIDS2017\cic_label_wordindex.json: {'benign': 1, 'ftppatator': 2, 'sshpatator': 3, 'dosslowloris': 4, 'dosslowhttptest': 5, 'doshulk': 6, 'dosgoldeneye': 7, 'heartbleed': 8, 'bruteforce': 9, 'xss': 10, 'sqlinjection': 11, 'infiltration': 12, 'bot': 13, 'portscan': 14, 'ddos': 15}


At this point, no non-number-stuff should remain in cic_data.  
Let's check.

In [11]:
cic_data.head()

,source_port,destination_port,protocol,timestamp,flow_duration,total_fwd_packets,total_backward_packets,total_length_of_fwd_packets,total_length_of_bwd_packets,fwd_packet_length_max,fwd_packet_length_min,fwd_packet_length_mean,fwd_packet_length_std,bwd_packet_length_max,bwd_packet_length_min,bwd_packet_length_mean,bwd_packet_length_std,flow_bytes_per_s,flow_packets_per_s,flow_iat_mean,flow_iat_std,flow_iat_max,flow_iat_min,fwd_iat_total,fwd_iat_mean,fwd_iat_std,fwd_iat_max,fwd_iat_min,bwd_iat_total,bwd_iat_mean,bwd_iat_std,bwd_iat_max,bwd_iat_min,fwd_psh_flags,bwd_psh_flags,fwd_urg_flags,bwd_urg_flags,fwd_header_length,bwd_header_length,fwd_packets_per_s,bwd_packets_per_s,min_packet_length,max_packet_length,packet_length_mean,packet_length_std,packet_length_variance,fin_flag_count,syn_flag_count,rst_flag_count,psh_flag_count,ack_flag_count,urg_flag_count,cwe_flag_count,ece_flag_count,down_per_up_ratio,average_packet_size,avg_fwd_segment_size,avg_bwd_segment_size,fwd_header_length.1,fwd_avg_bytes_per_bulk,fwd_avg_packets_per_bulk,fwd_avg_bulk_rate,bwd_avg_bytes_per_bulk,bwd_avg_packets_per_bulk,bwd_avg_bulk_rate,subflow_fwd_packets,subflow_fwd_bytes,subflow_bwd_packets,subflow_bwd_bytes,init_win_bytes_forward,init_win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,active_mean,active_std,active_max,active_min,idle_mean,idle_std,idle_max,idle_min,source_ip_o1,source_ip_o2,source_ip_o3,source_ip_o4,destination_ip_o1,destination_ip_o2,destination_ip_o3,destination_ip_o4,external_ip_o1,external_ip_o2,external_ip_o3,external_ip_o4
0,80,49188,6,2017-07-03 08:55:58,4,2,0,12.0,0.0,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,3000000.0,5.000000e+05,4.0,0.0,4.0,4.0,4.0,4.0,0.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,40,0,5.000000e+05,0.0,6.0,6.0,6.0,0.0,0.0,0,0,0,0,1,1,0,0,0.0,9.0,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,329,-1,1,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8,254,250,126,192,168,10,5,0,0,0,0
1,80,49188,6,2017-07-03 08:55:58,1,2,0,12.0,0.0,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,12000000.0,2.000000e+06,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,40,0,2.000000e+06,0.0,6.0,6.0,6.0,0.0,0.0,0,0,0,0,1,1,0,0,0.0,9.0,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,329,-1,1,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8,254,250,126,192,168,10,5,0,0,0,0
2,80,49188,6,2017-07-03 08:55:58,1,2,0,12.0,0.0,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,12000000.0,2.000000e+06,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,40,0,2.000000e+06,0.0,6.0,6.0,6.0,0.0,0.0,0,0,0,0,1,1,0,0,0.0,9.0,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,329,-1,1,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8,254,250,126,192,168,10,5,0,0,0,0
3,80,49188,6,2017-07-03 08:55:58,1,2,0,12.0,0.0,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,12000000.0,2.000000e+06,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,40,0,2.000000e+06,0.0,6.0,6.0,6.0,0.0,0.0,0,0,0,0,1,1,0,0,0.0,9.0,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,329,-1,1,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8,254,250,126,192,168,10,5,0,0,0,0
4,80,49486,6,2017-07-03 08:56:22,3,2,0,12.0,0.0,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,4000000.0,6.666667e+05,3.0,0.0,3.0,3.0,3.0,3.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,40,0,6.666667e+05,0.0,6.0,6.0,6.0,0.0,0.0,0,0,0,0,1,1,0,0,0.0,9.0,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,245,-1,1,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8,253,185,121,192,168,10,14,0,0,0,0


## Feature Standardization

As a matter of fact, sklearn doesn't like scaling timestamps (duuuh). So we split off the timestamp, normalize and append it again in the end.

In [12]:
timestamps = cic_data.filter(['timestamp'])
cic_data.drop(['timestamp'], axis=1, inplace=True)

As the scaler converts the DataFrames to numpy arrays, save the header info to recreate a DataFrame afterwards.

In [13]:
cic_data_header = list(cic_data.columns.values)

As many ML implementations behave badly if confronted with non-scaled inputs, we go ahead and transform all features to center, then scale it.  
**FIXME**: Please do not fit on concatenated train and test data! Thank you!

In [14]:
scaler = preprocessing.MinMaxScaler()
scaler.fit(cic_data) # fit the scaler on the training data

# transform samples without any refitting
cic_data = scaler.transform(cic_data)

In [15]:
# Recreate the DataFrame
cic_data = pd.DataFrame(columns=cic_data_header, data=cic_data)

In [16]:
# ... and append the timetamps again.
cic_data = pd.concat([cic_data,timestamps], axis=1)

Let's look at the scaled data

In [17]:
cic_data.head()

,source_port,destination_port,protocol,flow_duration,total_fwd_packets,total_backward_packets,total_length_of_fwd_packets,total_length_of_bwd_packets,fwd_packet_length_max,fwd_packet_length_min,fwd_packet_length_mean,fwd_packet_length_std,bwd_packet_length_max,bwd_packet_length_min,bwd_packet_length_mean,bwd_packet_length_std,flow_bytes_per_s,flow_packets_per_s,flow_iat_mean,flow_iat_std,flow_iat_max,flow_iat_min,fwd_iat_total,fwd_iat_mean,fwd_iat_std,fwd_iat_max,fwd_iat_min,bwd_iat_total,bwd_iat_mean,bwd_iat_std,bwd_iat_max,bwd_iat_min,fwd_psh_flags,bwd_psh_flags,fwd_urg_flags,bwd_urg_flags,fwd_header_length,bwd_header_length,fwd_packets_per_s,bwd_packets_per_s,min_packet_length,max_packet_length,packet_length_mean,packet_length_std,packet_length_variance,fin_flag_count,syn_flag_count,rst_flag_count,psh_flag_count,ack_flag_count,urg_flag_count,cwe_flag_count,ece_flag_count,down_per_up_ratio,average_packet_size,avg_fwd_segment_size,avg_bwd_segment_size,fwd_header_length.1,fwd_avg_bytes_per_bulk,fwd_avg_packets_per_bulk,fwd_avg_bulk_rate,bwd_avg_bytes_per_bulk,bwd_avg_packets_per_bulk,bwd_avg_bulk_rate,subflow_fwd_packets,subflow_fwd_bytes,subflow_bwd_packets,subflow_bwd_bytes,init_win_bytes_forward,init_win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,active_mean,active_std,active_max,active_min,idle_mean,idle_std,idle_max,idle_min,source_ip_o1,source_ip_o2,source_ip_o3,source_ip_o4,destination_ip_o1,destination_ip_o2,destination_ip_o3,destination_ip_o4,external_ip_o1,external_ip_o2,external_ip_o3,external_ip_o4,timestamp
0,0.001221,0.750561,0.352941,1.416667e-07,0.000005,0.0,9.302326e-07,0.0,0.000242,0.002581,0.00101,0.0,0.0,0.0,0.0,0.0,0.113208,0.416667,1.416667e-07,0.0,1.416667e-07,1.500000e-07,3.333333e-08,3.333333e-08,0.0,3.333333e-08,1.333333e-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.999856,0.994592,0.166667,0.0,0.004144,0.000242,0.001798,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.002312,0.00101,0.0,0.999856,0.0,0.0,0.0,0.0,0.0,0.0,0.000005,9.323764e-07,0.0,0.0,0.005035,0.0,0.000005,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.035874,0.996078,0.980392,0.494118,0.752941,0.658824,0.039216,0.019608,0.0,0.0,0.0,0.0,2017-07-03 08:55:58
1,0.001221,0.750561,0.352941,1.166667e-07,0.000005,0.0,9.302326e-07,0.0,0.000242,0.002581,0.00101,0.0,0.0,0.0,0.0,0.0,0.117067,0.666667,1.166667e-07,0.0,1.166667e-07,1.250000e-07,8.333333e-09,8.333333e-09,0.0,8.333333e-09,1.083333e-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.999856,0.994592,0.666667,0.0,0.004144,0.000242,0.001798,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.002312,0.00101,0.0,0.999856,0.0,0.0,0.0,0.0,0.0,0.0,0.000005,9.323764e-07,0.0,0.0,0.005035,0.0,0.000005,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.035874,0.996078,0.980392,0.494118,0.752941,0.658824,0.039216,0.019608,0.0,0.0,0.0,0.0,2017-07-03 08:55:58
2,0.001221,0.750561,0.352941,1.166667e-07,0.000005,0.0,9.302326e-07,0.0,0.000242,0.002581,0.00101,0.0,0.0,0.0,0.0,0.0,0.117067,0.666667,1.166667e-07,0.0,1.166667e-07,1.250000e-07,8.333333e-09,8.333333e-09,0.0,8.333333e-09,1.083333e-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.999856,0.994592,0.666667,0.0,0.004144,0.000242,0.001798,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.002312,0.00101,0.0,0.999856,0.0,0.0,0.0,0.0,0.0,0.0,0.000005,9.323764e-07,0.0,0.0,0.005035,0.0,0.000005,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.035874,0.996078,0.980392,0.494118,0.752941,0.658824,0.039216,0.019608,0.0,0.0,0.0,0.0,2017-07-03 08:55:58
3,0.001221,0.750561,0.352941,1.166667e-07,0.000005,0.0,9.302326e-07,0.0,0.000242,0.002581,0.00101,0.0,0.0,0.0,0.0,0.0,0.117067,0.666667,1.166667e-07,0.0,1.166667e-07,1.250000e-07,8.333333e-09,8.333333e-09,0.0,8.333333e-09,1.083333e-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.999856,0.994592,0.666667,0.0,0.004144,0.000242,0.001798,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.002312,0.00101,0.0,0.999856,0.0,0.0,0.0,0.0,0.0,0.0,0.000005,9.323764e-07,0.0,0.0,0.005035,0.0,0.000005,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.035874,0.996078,0.980392,0.494118,0.752941,0.658824,0.0392

## Serialization

So at this point, we have training and test sets with data and labels. The data parts are encoded and scaled, the encoded indizes are written away as json files.  
It would be nice if this data could be used for future runs, right? Right!  
That's why we serialize each dataframe into a python binary pickle on it's own (which is a feature directly supported by [Pandas](https://pandas.pydata.org/pandas-docs/stable/api.html#id12) - nice, eh?)

In [18]:
def write_to_pickle(dataframe, filename):
    dataframe.to_pickle(os.path.join('CICIDS2017', filename+'.pkl'))

In [19]:
write_to_pickle(cic_data, 'cic_data')
write_to_pickle(cic_labels, 'cic_labels')